# Create radial averages for pump-probe heatmap

In [1]:
%pylab qt

Populating the interactive namespace from numpy and matplotlib


### Imports and utility functions

In [2]:
from src.timepix_utils import *
import src.timepix_utils as timepix_utils

timepix_utils.file_system = 'beamline'

### Define source file & describe measurment

In [3]:
run_number = 775
timepix_event_type = 'raw'

hdf_file_name = generate_hdf_file_name(run_number)

tof_start = 0E-6
tof_end = 24E-6

### Complete time-of-flight

In [4]:
tof, x_pos, y_pos = data_sliced_by_tof(hdf_file_name, tof_start , tof_end, event_type = timepix_event_type)
plot_tof(tof, tof_start, tof_end,  hist_bins = 1000, time_unit = 'micro');

Number of raw events in the file: 9.127E+06
Number of FEL pulses in the file: 8.837E+03


### Slice time-of-flight dimension and create VMI image

In [5]:
fragment = 'I+'


display_tof_and_vmi_of_tof_interval(hdf_file_name, fragment, event_type = timepix_event_type,time_unit = 'micro')

Number of raw events in the file: 9.127E+06
Number of FEL pulses in the file: 8.837E+03


### Chose center and calc radial average

In [6]:
tof, x_pos, y_pos = data_sliced_by_fragment(hdf_file_name, fragment)

Number of raw events in the file: 9.127E+06
Number of FEL pulses in the file: 8.837E+03


In [7]:
radius_assisting_circle = 20

transform_vmi_to_polar(x_pos, y_pos, fragment, radius_assisting_circle);

## Save to intermediate results 

In [8]:
def save_all_delay_for_fragment_list(output_file_name, run_interval, fragment_list, timepix_event_type):
    runs_file , delay_stage = get_delay_stage_pos_from_txt_file(run_interval)
    with h5py.File('intermediate_data/'+output_file_name+'.hdf5', "w") as f:
        for i in range(run_interval[0], run_interval[1]+1):
            print('Working on Run {} ...{}/{}'.format(i,(i-run_interval[0])+1,(run_interval[1]+1-run_interval[0])))
            try:
                hdf_file = generate_hdf_file_name(i)
                number_of_trains = number_of_trains_from_hdf(hdf_file)
                run_hdfgroup = f.create_group("Run {}".format(i))
                run_hdfgroup.create_dataset('number_of_trains', data = number_of_trains)
                run_hdfgroup.create_dataset('delay_stage', data = delay_stage[i-run_interval[0]])
                for i in range(len(fragment_list)):             
                    tof, x_pos, y_pos = data_sliced_by_fragment(hdf_file, fragment_list[i], event_type = timepix_event_type)
                    radial_average = transform_vmi_to_polar(x_pos, y_pos, fragment_list[i], show_images = False) 
                    fragment_hdfgroup = run_hdfgroup.create_group("{}".format(fragment_list[i]))
                    fragment_hdfgroup.create_dataset('radial_average', data = radial_average)
            except:
                pass


In [14]:
timepix_event_type = 'raw'

output_file_name = '266nm3' 

#run_interval = [769, 794]
#run_interval = [769, 807]
run_interval = [808, 828]

fragment_list = ['C5H4N','I+','H+','C+','H2O+','I6+','C2Hn+','I4+','C3Hn+','I3+','I2+all','I2+highE']


In [15]:
save_all_delay_for_fragment_list(output_file_name, run_interval, fragment_list , timepix_event_type)


Working on Run 808 ...1/21
Number of raw events in the file: 9.748E+06
Number of FEL pulses in the file: 1.007E+04
Number of raw events in the file: 9.748E+06
Number of FEL pulses in the file: 1.007E+04
Number of raw events in the file: 9.748E+06
Number of FEL pulses in the file: 1.007E+04
Number of raw events in the file: 9.748E+06
Number of FEL pulses in the file: 1.007E+04
Number of raw events in the file: 9.748E+06
Number of FEL pulses in the file: 1.007E+04
Number of raw events in the file: 9.748E+06
Number of FEL pulses in the file: 1.007E+04
Number of raw events in the file: 9.748E+06
Number of FEL pulses in the file: 1.007E+04
Number of raw events in the file: 9.748E+06
Number of FEL pulses in the file: 1.007E+04
Number of raw events in the file: 9.748E+06
Number of FEL pulses in the file: 1.007E+04
Number of raw events in the file: 9.748E+06
Number of FEL pulses in the file: 1.007E+04
Number of raw events in the file: 9.748E+06
Number of FEL pulses in the file: 1.007E+04
Numbe